In [1]:
# do something like this - copy over from fastai2 walkthru 
# !nvidia-smi -L

In [2]:
# make your Google drive accessible 
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"
base_dir = root_dir + 'fastai2_library/course-v4/'

# navigate to the notebooks directory for dl2
import os
os.chdir(base_dir)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
!pwd
# cd to base_dir if above os.chdir does not work using below command
# %cd "/content/gdrive/My Drive/fastai/course-v4/"

/content/gdrive/My Drive/fastai2_library/course-v4


In [4]:
!pip install -q -r requirements.txt

     |████████████████████████████████| 184kB 2.8MB/s 
     |████████████████████████████████| 61kB 4.9MB/s 
     |████████████████████████████████| 51kB 2.4MB/s 
     |████████████████████████████████| 1.1MB 8.4MB/s 
     |████████████████████████████████| 40kB 5.2MB/s 
     |████████████████████████████████| 92kB 8.0MB/s 
     |████████████████████████████████| 61kB 6.7MB/s 
     |████████████████████████████████| 51kB 5.9MB/s 
     |████████████████████████████████| 2.7MB 17.4MB/s 


In [5]:
%cd nbs

/content/gdrive/My Drive/fastai2_library/course-v4/nbs


In [ ]:
#default_exp dispatch

In [6]:
#export
from fastcore.imports import *
from fastcore.foundation import *
from fastcore.utils import *

In [7]:
# from nbdev.showdoc import *
from fastcore.test import *

# Type dispatch

> Basic single and dual parameter dispatch

## Helpers

In [8]:
#exports
# modified version of Python typing.typehints as per below i.e., {} when not allowed type
def type_hints(f):
    "Same as `typing.get_type_hints` but returns `{}` if not allowed type"
    return typing.get_type_hints(f) if isinstance(f, typing._allowed_types) else {}

In [ ]:
typing.get_type_hints??

In [9]:
#export
def anno_ret(func):
    "Get the return annotation of `func`"
    if not func: return None
    ann = type_hints(func)
    if not ann: return None
    return ann.get('return')

In [10]:
#hide
def f(x:Int) -> float: return x
test_eq(anno_ret(f), float)
def f(x) -> float: return x
test_eq(anno_ret(f), float)
def f(x) -> typing.Tuple[float,float]: return x
test_eq(anno_ret(f), typing.Tuple[float,float])
# Notice the two following - one case NoneType other case None
# Think helps to distinguish return input type when None
# and return no type at all for output when NoneType.
# Also None when no func

# here output return type specified as None
# so (think) return type that Python returns
def f(x) -> None: return x
test_eq(anno_ret(f), NoneType)

# here output return no Type at all 
# so return whatever input type you got
def f(x): return x
test_eq(anno_ret(f), None)
test_eq(anno_ret(None), None)

In [11]:
#export

# cmp_to_key is passed in func which creates key for sorting
# here -1 if a not subclass of b, if a is subclass of b then 0 if a==b, 1 otherwise.
cmp_instance = functools.cmp_to_key(lambda a,b: 0 if a==b else 1 if issubclass(a,b) else -1)

In [12]:
# Check link below for class hierarchy of numbers
# https://docs.python.org/2/library/numbers.html
td = {int:1, numbers.Number:2, numbers.Integral:3}
test_eq(sorted(td, key=cmp_instance), [numbers.Number, numbers.Integral, int])

In [13]:
#export
def _p2_anno(f):
    "Get the 1st 2 annotations of `f`, defaulting to `object`"
    hints = type_hints(f)
    # here only looking at input annotations NOT return annotations
    ann = [o for n,o in hints.items() if n!='return']
    # if 2nd annotation does not exist make it an object
    while len(ann)<2: ann.append(object)
    return ann[:2]

In [14]:
def _f(a): pass
test_eq(_p2_anno(_f), (object,object))
def _f(a, b): pass
test_eq(_p2_anno(_f), (object,object))
def _f(a:None, b)->str: pass
test_eq(_p2_anno(_f), (NoneType,object))
def _f(a:str, b)->float: pass
test_eq(_p2_anno(_f), (str,object))
def _f(a:None, b:str)->float: pass
test_eq(_p2_anno(_f), (NoneType,str))
#func with 2 inputs
def _f(a:int, b:int)->float: pass
test_eq(_p2_anno(_f), (int,int))
#method with 2 inputs
def _f(self, a:int, b:int): pass
test_eq(_p2_anno(_f), (int,int))
def _f(a:int, b:str)->float: pass
test_eq(_p2_anno(_f), (int,str))
test_eq(_p2_anno(attrgetter('foo')), (object,object))

## TypeDispatch -

The following class is the basis that allows us to do type dipatch with type annotations. It contains a dictionary type -> functions and ensures that the proper function is called when passed an object (depending on its type).

In [20]:
#export
class _TypeDict:
    def __init__(self): self.d,self.cache = {},{}

    def _reset(self):
        # see skr_04_transform nb sorted eg use cmp_instance - the rightmost is the
        # most specific type of subclass eg int and hence need for reverse=True
        # if you are grabbing res[0] below.
        # self.d contains ordered subclasses with lowest matching first
        self.d = {k:self.d[k] for k in sorted(self.d, key=cmp_instance, reverse=True)}
        self.cache = {}

    def add(self, t, f):
        "Add type `t` and function `f`"
        if not isinstance(t,tuple): t=tuple(L(t))
        for t_ in t: self.d[t_] = f
        self._reset()

    def all_matches(self, k):
        "Find first matching type that is a super-class of `k`"
        if k not in self.cache:
            # create a list of all of the types that are registered for which there is
            # the appropriate subclass relationship and notice that func in self.d
            # has been ordered by a comparator which checks for subclass relationship
            # and we grab first one res[0] and that is the most specific type of the
            # ones that it is a subclass of.
            types = [f for f in self.d if k==f or (isinstance(k,type) and issubclass(k,f))]
            # add to cache once we find it since this search takes time and we need
            # to do this every time we call this function so when we find the right one
            # we store in cache so next time around we can just grab it.
            self.cache[k] = [self.d[o] for o in types]
        return self.cache[k]

    def __getitem__(self, k):
        "Find first matching type that is a super-class of `k`"
        res = self.all_matches(k)
        return res[0] if len(res) else None

    def __repr__(self): return self.d.__repr__()
    def first(self): return first(self.d.values())

It needs to work a lot like a dict because we will be adding things to it.

Recall that encodes, decodes, setups of Transforms are objects of type TypeDispatch. So we want to dispatch different code depending on type. (See
Rotate example below).

Dispatch refers to how a programming language decides which piece of code to run when you call something. Different languages do this in different ways.

We want something for TypeDispatch that looks like a function so has a \_\_call\_\_
but when you call it with some argument, we are not just going to call a function BUT we are going to look at the type of the argument and we are going to find the appropriate function or method to call based on the type of the argument passed in and the methods created so far. 

So, inside our TypeDispatch is a dict called funcs where the key is the type so for eg in Rotate eg below we have keys, TensorImage and Bbox, and the values are the functions to call for each key. 

There is an add method in TypeDispatch which is what _TfmDict calls.It adds this function - when it does self[k].add(v). The add method finds the type annotation using _p2_anno and if anno is None assumes it is object (which is highest level of the type hierarchy) and it pops it into our funcs dictionary.


Later when we call \_\_call\_\_ it is going to look up the type of parameter you are calling this function on by mapping type (see below) and then it is going to look it up in this object. If it does not find it, it returns arg[0] which implies doing nothing but just returning x (so x is arg[0]). So if you call Rotate with Int (instead of with TensorImage or BoundingBox) nothing happens implies it return whatever is its input unchanged. That function is not defined for that type. If we do find it we call it with anything else you passed along (*args, **kwargs). Can turn into method if needed.

How does it look up type? See \_\_getitem\_\_ code 

We keep a cache in _TypeDict which is a dict mapping from types to functions. We need a special cache dict coz of way TypeDispatch works (explained below). If we do not find it in cache - self.func.all_matches(k[0]) - if k is not in cache (in all_matches code) then we add it to cache as per code in all_matches method. 

TypeDispatch looks not just at type of TensorImage, BoundingBox etc but looks at subclassess of those for e.g., if we also add Rotate for Tensors TypeDispatch
will grab the most specific (lowest subclass) version it can which is a TensorImage. Also if you have defined encodes on Tensor and call it on TensorImage which is subclass of Tensor, then it WILL be invoked because of the way the cache is created. 



In [ ]:
# Eg
# class Rotate(Transform): pass

In [ ]:
# @Rotate
# def encodes(x:Tensor, deg): ...

In [ ]:
# @Rotate
# def encodes(x:TensorImage, deg): ...

In [ ]:
# @Rotate
# def encodes(x:BoundingBox, deg): ...

In [21]:
#export
class TypeDispatch:
    "Dictionary-like object; `__getitem__` matches keys of types using `issubclass`"
    def __init__(self, funcs=(), bases=()):
        self.funcs,self.bases = _TypeDict(),L(bases).filter(is_not(None))
        for o in L(funcs): self.add(o)
        self.inst = None

    def add(self, f):
        "Add type `t` and function `f`"
        # get type annotations for first 2 params
        a0,a1 = _p2_anno(f)
        t = self.funcs.d.get(a0)
        if t is None:
            t = _TypeDict()
            # TypeDict add
            self.funcs.add(a0, t)
        # Below is TypeDict add.
        t.add(a1, f)

    def first(self): return self.funcs.first().first()
    def returns(self, x): return anno_ret(self[type(x)])
    def returns_none(self, x):
        r = anno_ret(self[type(x)])
        return r if r == NoneType else None

    def _attname(self,k): return getattr(k,'__name__',str(k))
    def __repr__(self):
        r = [f'({self._attname(k)},{self._attname(l)}) -> {getattr(v, "__name__", v.__class__.__name__)}'
             for k in self.funcs.d for l,v in self.funcs[k].d.items()]
        return '\n'.join(r)

    def __call__(self, *args, **kwargs):
        ts = L(args).map(type)[:2]
        # The line below is basically a call to __getitem__
        f = self[tuple(ts)]
        if not f: return args[0]
        # turn it into method instead of normal func - will discuss details later
        # checks for self.inst - finds out whether this is an instance of something
        # if it is (the check self.inst is not None) then instead of using function
        # looked up, it wraps it as a method. So a method: where the method is the 
        # function f and the instance the function is bound to is self.inst
        # This is how you turn a function into a method.
        # How does it know what self.inst is? (in eg self.inst set to a)
        # See text box below
        if self.inst is not None: f = MethodType(f, self.inst)
        return f(*args, **kwargs)

    def __get__(self, inst, owner):
        self.inst = inst
        return self

    def __getitem__(self, k):
        "Find first matching type that is a super-class of `k`"
        k = L(k)
        while len(k)<2: k.append(object)
        # self.funcs is _TypeDict so it uses all_matches method 
        r = self.funcs.all_matches(k[0])
        for t in r:
            o = t[k[1]]
            if o is not None: return o
        for base in self.bases:
            res = base[k]
            if res is not None: return res
        return None

How does TypeDispatch know what self.inst is? (Brief ans: It is a descriptor)
In Python when you say a.f Python calls a special method \_\_get\_\_ of whatever class F is in. (Refer to Python Data Model Section 3.3.2 - Customizing Attribute Access or more precisely 3.3.2.2 Implementing descriptors). So \_\_get\_\_ in TypeDispatch is called, and it is going to be passed in as inst the instance a of owner A. By doing self.inst = inst, from that point, the TypeDispatch object knows what instance it is being called on. So when self.inst is not None -> there is an instance, we wrap f as a method of the instance. 

With this mod, TypeDispatch handles functions and methods BOTH automagically!!

Some Descriptor reference links:

DOCS:

https://realpython.com/python-descriptors/
https://nbviewer.jupyter.org/urls/gist.github.com/ChrisBeaumont/5758381/raw/descriptor_writeup.ipynb#Accessing-a-descriptor
https://docs.python.org/3/reference/datamodel.html#implementing-descriptors
https://docs.python.org/3/howto/descriptor.html

VIDEOS:

https://www.youtube.com/watch?v=NiSqG6s8skA&t=126s

https://www.youtube.com/watch?v=81S01c9zytE&t=16s

https://www.youtube.com/watch?v=7PzeZQGVPKc&list=PLLj6w0Thbv00Ij2yYC5PJOUaD_xuQcm7O&index=33&t=14s



Let us look at some examples. First I am just exploring what _p2_anno(f) returns for various f passed into it.

In [ ]:
def f_ni2(x:int): return x
a0, a1 = _p2_anno(f_ni2)
print(a0)
print(type(a0))
print(a1)
print(type(a1))

<class 'int'>
<class 'type'>
<class 'object'>
<class 'type'>


In [ ]:
def f_nin(x:numbers.Integral)->int:  return x+1
a0, a1 = _p2_anno(f_nin)
print(a0)
print(type(a0))
print(a1)
print(type(a1))

<class 'numbers.Integral'>
<class 'abc.ABCMeta'>
<class 'object'>
<class 'type'>


In [ ]:
def f_bll(x:(bool,list)): return x
a0, a1 = _p2_anno(f_bll)
print(a0)
print(type(a0))
print(a1)
print(type(a1))

(<class 'bool'>, <class 'list'>)
<class 'tuple'>
<class 'object'>
<class 'type'>


In [ ]:
def f_num(x:numbers.Number): return x
a0, a1 = _p2_anno(f_num)
print(a0)
print(type(a0))
print(a1)
print(type(a1))

<class 'numbers.Number'>
<class 'abc.ABCMeta'>
<class 'object'>
<class 'type'>


Now let us look at what a TypeDispatch object DOES and what it IS

In [22]:
def f_col(x:typing.Collection): return x
def f_nin(x:numbers.Integral)->int:  return x+1
def f_ni2(x:int): return x
def f_num(x:numbers.Number): return x
def f_bll(x:(bool,list)): return x

t = TypeDispatch([f_nin,f_ni2,f_num,f_bll,None])

t.add(f_ni2) #Should work even if we add the same function twice.
# For below keys are types, values are functions
# so t[key] points to appropriate function (to dispatch)

# treat t or TypeDispatch object like a dict passing type which
# tells us what function we should call for that type
# int type match hence f_ni2
test_eq(t[int], f_ni2)
# numbers.Integral a closer match than numbers.Number hence f_nin
test_eq(t[np.int32], f_nin)
# no str type hence None
test_eq(t[str], None)
test_eq(t[float], f_num)
test_eq(t[bool], f_bll)
test_eq(t[list], f_bll)
t.add(f_col)
test_eq(t[str], f_col)
test_eq(t[np.int32], f_nin)
o = np.int32(1)
# note that below is call to the TypeDispatch object so t(o) not 
# t[0] which returns a func as we see above. Below call returns
# the func (a callable) run on argument np.int32(1) 
# hence with typeDispatch returns x+1
test_eq(t(o), 2)
test_eq(t.returns(o), int)
assert t.first() is not None
# Print a TypeDispatch object - uses __repr__  
t

(list,object) -> f_bll
(Collection,object) -> f_col
(bool,object) -> f_bll
(int,object) -> f_ni2
(Integral,object) -> f_nin
(Number,object) -> f_num
(object,object) -> NoneType

If `bases` is set to a collection of `TypeDispatch` objects, then they are searched matching functions if no match is found in this object.

This is what the: for base in self.bases code in the \_\_getitem\_\_ method does

In [23]:
def f_str(x:str): return x+'1'

t2 = TypeDispatch(f_str, bases=t)
test_eq(t2[int], f_ni2)
test_eq(t2[np.int32], f_nin)
test_eq(t2[float], f_num)
test_eq(t2[bool], f_bll)
test_eq(t2[str], f_str)
test_eq(t2('a'), 'a1')
test_eq(t2[np.int32], f_nin)
test_eq(t2(o), 2)
test_eq(t2.returns(o), int)

Notice that below functions use self as first argument unlike simple functions above. 

In [24]:
def m_nin(self, x:(str,numbers.Integral)): return str(x)+'1'
def m_bll(self, x:bool): self.foo='a'
def m_num(self, x:numbers.Number): return x

# Create TypeDispatch object as usual
t = TypeDispatch([m_nin,m_num,m_bll])

# Insert the TypeDispatch object as an attribute of 
# class A and call the attribute f
# Now we call a.f and pass in 1 and get back '11'
class A: f = t
a = A()
# 1 converted to str and +'1' as 1 matches numbers.integral
test_eq(a.f(1), '11')

# Bit of magic. How did Python/TypeDispatch know that a.f should
# be passed self as well as the X. By default it does NOT.
# There is nothing that says a.f which is just an attribute of a class
# should be passed a self - so HOW? see __call__ of TypeDispatch.

# float 1. no match so returns input
test_eq(a.f(1.), 1.)
# method check - will return to this
# that is it is somehow binding a self to this object a.f
test_is(a.f.inst, a)
a.f(False)
# since boolean passed, a.foo set to 'a'
test_eq(a.foo, 'a')
# When you pass in no func nothing returned??
test_eq(a.f(()), ())

In [25]:
def m_tup(self, x:tuple): return x+(1,)
t2 = TypeDispatch(m_tup, t)
class A2: f = t2
a2 = A2()
test_eq(a2.f(1), '11')
test_eq(a2.f(1.), 1.)
test_is(a2.f.inst, a2)
a2.f(False)
test_eq(a2.foo, 'a')
test_eq(a2.f(()), (1,))

In [26]:
def f1(x:numbers.Integral, y): return x+1
def f2(x:int, y:float): return x+y
t = TypeDispatch([f1,f2])

test_eq(t[int], f1)
test_eq(t[int,int], f1)
test_eq(t[int,float], f2)
test_eq(t[float,float], None)
# below is the interesting one
# not f2 but f1 coz first anno only matches f1
test_eq(t[np.int32,float], f1)
test_eq(t(3,2.0), 5)
test_eq(t(3,2), 4)
test_eq(t('a'), 'a')
t

(int,float) -> f2
(Integral,object) -> f1

## typedispatch Decorator

In [ ]:
#export
class DispatchReg:
    "A global registry for `TypeDispatch` objects keyed by function name"
    def __init__(self): self.d = defaultdict(TypeDispatch)
    def __call__(self, f):
        nm = f'{f.__qualname__}'
        self.d[nm].add(f)
        return self.d[nm]

typedispatch = DispatchReg()

In [ ]:
@typedispatch
def f_td_test(x, y): return f'{x}{y}'
@typedispatch
def f_td_test(x:numbers.Integral, y): return x+1
@typedispatch
def f_td_test(x:int, y:float): return x+y

test_eq(f_td_test(3,2.0), 5)
test_eq(f_td_test(3,2), 4)
test_eq(f_td_test('a','b'), 'ab')

## Casting

Now that we can dispatch on types, let's make it easier to cast objects to a different type.

In [ ]:
#export
_all_=['cast']

In [ ]:
#export
def retain_meta(x, res, copy_meta=False):
    "Call `res.set_meta(x)`, if it exists"
    if hasattr(res,'set_meta'): res.set_meta(x, copy_meta=copy_meta)
    return res

In [ ]:
#export
def default_set_meta(self, x, copy_meta=False):
    "Copy over `_meta` from `x` to `res`, if it's missing"
    if hasattr(x, '_meta') and not hasattr(self, '_meta'):
        meta = x._meta
        if copy_meta: meta = copy(meta)
        self._meta = meta
    return self

In [ ]:
#export
@typedispatch
def cast(x, typ):
    "cast `x` to type `typ` (may also change `x` inplace)"
    res = typ._before_cast(x) if hasattr(typ, '_before_cast') else x
    if isinstance(res, ndarray): res = res.view(typ)
    elif hasattr(res, 'as_subclass'): res = res.as_subclass(typ)
    else:
        try: res.__class__ = typ
        except: res = typ(res)
    return retain_meta(x, res)

NameError: ignored

This works both for plain python classes:...

In [ ]:
mk_class('_T1', 'a')
class _T2(_T1): pass

t = _T1(a=1)
t2 = cast(t, _T2)
test_eq_type(_T2(a=1), t2)

...as well as for arrays and tensors.

In [ ]:
class _T1(ndarray): pass

t = array([1])
t2 = cast(t, _T1)
test_eq(array([1]), t2)
test_eq(_T1, type(t2))

To customize casting for other types, define a separate `cast` function with `typedispatch` for your type.

In [ ]:
#export
def retain_type(new, old=None, typ=None, copy_meta=False):
    "Cast `new` to type of `old` or `typ` if it's a superclass"
    # e.g. old is TensorImage, new is Tensor - if not subclass then do nothing
    if new is None: return
    assert old is not None or typ is not None
    if typ is None:
        if not isinstance(old, type(new)): return new
        typ = old if isinstance(old,type) else type(old)
    # Do nothing the new type is already an instance of requested type (i.e. same type)
    if typ==NoneType or isinstance(new, typ): return new
    return retain_meta(old, cast(new, typ), copy_meta=copy_meta)

In [ ]:
class _T(tuple): pass
a = _T((1,2))
b = tuple((1,2))
test_eq_type(retain_type(b, typ=_T), a)

If `old` has a `_meta` attribute, its content is passed when casting `new` to the type of `old`.

In [ ]:
class _A():
    set_meta = default_set_meta
    def __init__(self, t): self.t=t

class _B1(_A):
    def __init__(self, t, a=1):
        super().__init__(t)
        self._meta = {'a':a}
        
x = _B1(1, a=2)
b = _A(1)
test_eq(retain_type(b, old=x)._meta, {'a': 2})

In [ ]:
a = {L: [int, tuple]}
first(a.keys())

In [ ]:
#export
def retain_types(new, old=None, typs=None):
    "Cast each item of `new` to type of matching item in `old` if it's a superclass"
    if not is_listy(new): return retain_type(new, old, typs)
    if typs is not None:
        if isinstance(typs, dict):
            t = first(typs.keys())
            typs = typs[t]
        else: t,typs = typs,None
    else: t = type(old) if old is not None and isinstance(old,type(new)) else type(new)
    return t(L(new, old, typs).map_zip(retain_types, cycled=True))

In [ ]:
class T(tuple): pass

t1,t2 = retain_types((1,(1,(1,1))), (2,T((2,T((3,4))))))
test_eq_type(t1, 1)
test_eq_type(t2, T((1,T((1,1)))))

t1,t2 = retain_types((1,(1,(1,1))), typs = {tuple: [int, {T: [int, {T: [int,int]}]}]})
test_eq_type(t1, 1)
test_eq_type(t2, T((1,T((1,1)))))

In [ ]:
#export
def explode_types(o):
    "Return the type of `o`, potentially in nested dictionaries for thing that are listy"
    if not is_listy(o): return type(o)
    return {type(o): [explode_types(o_) for o_ in o]}

In [ ]:
test_eq(explode_types((2,T((2,T((3,4)))))), {tuple: [int, {T: [int, {T: [int,int]}]}]})

## Export -

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_test.ipynb.
Converted 01_foundation.ipynb.
Converted 02_utils.ipynb.
Converted 03_dispatch.ipynb.
Converted 04_transform.ipynb.
Converted index.ipynb.
